In [122]:
'''
This module contains all functions relating to the cleaning, exploration and feature engineering of structured data sets; mostly in pandas format


install the anaconda python distribution to be able to use numpy, pandas, matplotlib, seaborn, scipy, sklearn and datetime

use pip install category_encoders for category_encoders 
use pip install category_embedder for category_embedder. It requires the tensorflow and keras
'''

##pip install category_encoders

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from category_encoders import *
from IPython.display import display
from collections import Counter
import scipy.stats as sp
import datetime as dt
import re
#import categorical_embedder
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, RobustScaler, StandardScaler
le = LabelEncoder()

## Exploratory and Statistical Analysis

In [72]:
def get_cat_feats(data=None):
    '''
    Returns the categorical features in a data set
    Parameters:
    -----------
        data: DataFrame or named Series 
    Returns:
    -------
        List
            A list of all the categorical features in a dataset.
    it is used as a helper function for most of the functions to get categorical variables
    '''
    if data is None:
        raise ValueError("data: Expecting a DataFrame or Series, got 'None'")

    cat_features = data.select_dtypes(include=['object']).columns

    return list(cat_features)

In [73]:
def get_num_feats(data=None):
    '''
    Returns the numerical features in a data set
    Parameters:
    -----------
        data: DataFrame or named Series 
    Returns:
    -------
        List:
            A list of all the numerical features in a dataset.
    it is used as a helper function for most of the functions to get categorical variables
    '''
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

    if data is None:
        raise ValueError("data: Expecting a DataFrame or Series, got 'None'")

    num_features = data.select_dtypes(include=numerics).columns

    return list(num_features)

In [74]:
def get_unique_counts(data=None):
    '''
    Gets the unique count of categorical features in a data set.
    Parameters
    -----------
        data: DataFrame or named Series 
    Returns
    -------
        DataFrame or Series
            Unique value counts of the features in a dataset.
    it is used as a helper function in the describe function to get the count of unique values in the columns 
    '''

    if data is None:
        raise ValueError("data: Expecting a DataFrame or Series, got 'None'")

    features = get_cat_feats(data)
    temp_len = []

    for feature in features:
        temp_len.append(len(data[feature].unique()))
        
    df = list(zip(features, temp_len))
    df = pd.DataFrame(df, columns=['Feature', 'Unique Count'])
    df = df.style.bar(subset=['Unique Count'], align='mid')
    return df

In [5]:
def display_missing(data=None, plot=False):
    '''
    Display missing values as a pandas dataframe.
    Parameters
    ----------
        data: DataFrame or named Series
        plot: bool, Default False
            Plots missing values in dataset as a heatmap
    
    Returns
    -------
        Matplotlib Figure:
            Heatmap plot of missing values
    '''

    if data is None:
        raise ValueError("data: Expecting a DataFrame or Series, got 'None'")

    df = data.isna().sum()
    df = df.reset_index()
    df.columns = ['features', 'missing_counts']

    missing_percent = round((df['missing_counts'] / data.shape[0]) * 100, 1)
    df['missing_percent'] = missing_percent

    if plot:
        plot_missing(data)
        return df
    else:
        return df

In [6]:
def cat_summarizer(data, x=None, y=None, hue=None, palette='Set1', verbose=True):
    '''
    Helper function that gives a quick summary of a given column of categorical data
    Parameters:
    ---------------------------
        dataframe: pandas dataframe
        x: str.
            horizontal axis to plot the labels of categorical data, y would be the count.
        y: str. 
            vertical axis to plot the labels of categorical data, x would be the count.
        hue: str. i
            if you want to compare it another variable (usually the target variable)
        palette: array, list.
            Colour of the plot
    Returns:
    ----------------------
        Quick Stats of the data and also the count plot
        
        it is used in the describe function
    '''
    if x == None:
        column_interested = y
    else:
        column_interested = x
    series = data[column_interested]
    print(series.describe())
    print('mode: ', series.mode())
    if verbose:
        print('='*80)
        print(series.value_counts())

    sns.countplot(x=x, y=y, hue=hue, data=data, palette=palette)
    plt.show()

In [7]:
def _space():
    '''it is used in  most functions to add space. this makes result more presentation'''
    print('\n')
def _match_date(data):
    '''
        Return a list of columns that matches the DateTime expression
    '''
    mask = data.sample(20).astype(str).apply(lambda x : x.str.match(r'(\d{2,4}-\d{2}-\d{2,4})+').all())
    return set(data.loc[:, mask].columns)


def display_rows(data,num=2):
    '''
    Displays the required number of rows
    it is used in the describe function
    '''
    if data is None:
        raise ValueError("data: Expecting a DataFrame/ numpy2d array, got 'None'")

    return data.head(num)

In [8]:
def plot_missing(data=None):
    '''
    Plots the data as a heatmap to show missing values
    Parameters
    ----------
        data: DataFrame, array, or list of arrays.
            The data to plot.
    '''

    if data is None:
        raise ValueError("data: Expecting a DataFrame or Series, got 'None'")

    sns.heatmap(data.isnull(), cbar=True)
    plt.show()

In [9]:
def class_count(data=None, features=None, plot=False, save_fig=False):
    '''
    Displays the number of classes in a categorical feature.
    Parameters:
    
        data: Pandas DataFrame or Series
            Dataset for plotting.
        features: Scalar, array, or list. 
            The categorical features in the dataset, if None, 
            we try to infer the categorical columns from the dataframe.
        plot: bool, Default False.
            Plots the class counts as a barplot
        save_fig: bool, Default False.
            Saves the plot to the current working directory.
    it is used in the describe function
    '''

    if data is None:
        raise ValueError("data: Expecting a DataFrame or Series, got 'None'")

    if features is None:
        features = get_cat_feats(data)

                        

    for feature in features:
        if data[feature].nunique() > 15:
            print("Unique classes in {} too large".format(feature))
        else:
            print('Class Count for', feature)
            display(pd.DataFrame(data[feature].value_counts()))

    if plot:
        countplot(data, features, save_fig=save_fig)

In [10]:
def get_date_cols(data=None):
    '''
    Returns the Datetime columns in a data set.
    Parameters
    ----------
        data: DataFrame or named Series
            Data set to infer datetime columns from.
        convert: bool, Default True
            Converts the inferred date columns to pandas DateTime type
    Returns:
    -------
        List
         Date column names in the data set
    use in the describe function to set date columns to datetime datatype in utc
    '''

    if data is None:
        raise ValueError("data: Expecting a DataFrame or Series, got 'None'")

    #Get existing date columns in pandas Datetime64 format
    date_cols = set(data.dtypes[data.dtypes == 'datetime64[ns, UTC]'].index)
    #infer Date columns 
    date_cols = date_cols.union(_match_date(data))
       
    return date_cols

In [11]:
def bivariate_stats(data):
    '''Returns the contingency table and chi2 contingency test result between columns in the dataframe
        
        it is used in the describe function for categorical features analysis 
    
    
    '''
    cat_feats = get_cat_feats(data=data)
    counter = 1
    try:
        while counter<(len(cat_feats)):
            val1 = get_cat_feats(data=datainput)[counter - 1]
            val2 = get_cat_feats(data=datainput)[counter]
            if (data[val1].nunique() > 15) or (data[val2].nunique() > 15):
                print('Number of unique values too large')
            else:
                freqtab = pd.crosstab(data[val1], data[val2])
                print("Frequency table")
                print("============================")
                print(freqtab)
                print("============================")
                chi2, pval, dof, expected = sp.chi2_contingency(freqtab)
                print("ChiSquare test statistic: ",chi2)
                print("p-value: ",pval)
                _space()
            counter= counter+1
    except:
        pass

In [12]:
def bivariate_stats_target(data, target):
    
    '''Returns the contingency table and chi2 contingency test result between columns and the target variable in the dataframe
        
        
        Parameters
    ----------
        data: DataFrame or named Series
            Data set to infer datetime columns from.
        target: the target variable in form of string

        
        it is used in the describe function for categorical features analysis of the relationship between the target variable 
        and other categorical features
    
    
    '''
    cat_feats = get_cat_feats(data=data)
    for i in cat_feats:
        if (data[i].nunique() > 20):
            print('Number of Unique values too large')
        else:
            freqtab = pd.crosstab(data[i], data[target])
            print("Frequency table")
            print("============================")
            print(freqtab)
            print("============================")
            chi2, pval, dof, expected = sp.chi2_contingency(freqtab)
            print("ChiSquare test statistic: ",chi2)
            print("p-value: ",pval)
            _space()

In [13]:
def describe(data=None, name='', date_cols=None, show_categories=False, plot_missing=False, target = None):
    '''
    Calculates statistics and information about a data set. Information displayed are
    shapes, size, number of categorical/numeric/date features, missing values,
    dtypes of objects, correlation analysis, contigency analysis etc.
    Parameters:
    --------------------
        data: Pandas DataFrame
            The data to describe.
        name: str, optional
            The name of the data set passed to the function.
        date_cols: list/series/array
            Date column names in the data set.
        show_categories: bool, default False
            Displays the unique classes and counts in each of the categorical feature in the data set.
        plot_missing: bool, default True
            Plots missing values as a heatmap
        target: the target variable in the dataframe
    Returns:
    -------
        None
        
        This function is stand alone use for quick statistical exploration of the data.
    '''
    
    if data is None:
        raise ValueError("data: Expecting a DataFrame or Series, got 'None'")

    ## Get categorical features
    cat_features = get_cat_feats(data)
    
    #Get numerical features
    num_features = get_num_feats(data)

    print('First five data points')
    display(data.head())
    _space()

    print('Random five data points')
    display(data.sample(5))
    _space()

    print('Last five data points')
    display(data.tail())
    _space()

    print('Shape of {} data set: {}'.format(name, data.shape))
    _space()

    print('Size of {} data set: {}'.format(name, data.size))
    _space()

    print('Data Types')
    print("Note: All Non-numerical features are identified as objects in pandas")
    display(pd.DataFrame(data.dtypes, columns=['Data Type']))
    _space()
    
    date_cols = get_date_cols(data)
    if len(date_cols) is not 0:
        print("Column(s) {} should be in Datetime format. Use the [to_date] function to convert to Pandas Datetime format".format(date_cols))
        _space()

    print('Numerical Features in Data set')
    print(num_features)
    _space()

    print('Categorical Features in Data set')
    display(cat_features)
    _space()

    print('Statistical Description of Columns')
    display(data.describe())
    _space()
    
    print('Description of Categorical Features')
    if cat_features != None:
        display(data.describe(include=[np.object, pd.Categorical]).T)
        _space()
          
    print('Unique class Count of Categorical features')
    display(get_unique_counts(data))
    _space()

    if show_categories:     
        print('Classes in Categorical Columns')
        print("-"*30)
        class_count(data, cat_features)
        _space()

    print('Missing Values in Data')
    display(display_missing(data))
    _space()
  
    print('Pearson Correlation')
    print(data.corr())
    _space()
    
    print('Kendall Correlation')
    print(data.corr(method='kendall'))
    _space()
    
    print('Spearman Correlation')
    print(data.corr(method='spearman'))
    _space()
    
    print('Bivariant Stats between categorical features')
    print(bivariate_stats(data))
    _space()
    if target is not None:
        print('Bivariant Stats between cat feats and target variable')
        print(bivariate_stats_target(data, target))
        _space()
    print('')

## Feature Processing; Missing Value Imputation, Date Feature Extraction and Categorical Feature Encoding.

In [14]:
def date_processing(data=None, date_cols = None, utc=True):
    '''
    This function is used for preprocessing date columns and creating new columns from the date columns.
    
    It is used in the feature_processing function.
    
    
    '''
    if data is None:
        raise ValueError("data: Expecting a DataFrame/ numpy2d array, got 'None'")
        
    for i in date_cols:
        data[i] = pd.to_datetime(data[i], utc=utc)
        data[f'month_{i}'] = data[i].dt.month
        data[f'year_{i}'] = data[i].dt.year
        data[f'day_{i}'] = data[i].dt.day
        data[f'dayweek_{i}'] = data[i].dt.weekday
    return data

In [15]:
def drop_missing(data=None, percent=99):
    '''
    Drops missing columns with [percent] of missing data.
    Parameters:
    -------------------------
        data: Pandas DataFrame or Series.
        percent: float, Default 99
            Percentage of missing values to be in a column before it is eligible for removal.
    Returns:
    ------------------
        Pandas DataFrame or Series.
    It can be used alone. It also used in deal_with_missing_value function.
    
    This function is used in the deal_with_missing_value function.
    '''

    if data is None:
        raise ValueError("data: Expecting a DataFrame/ numpy2d array, got 'None'")
    
    missing_percent = (data.isna().sum() / data.shape[0]) * 100
    cols_2_drop = missing_percent[missing_percent.values >= percent].index
    print("Dropped {}".format(list(cols_2_drop)))
    #Drop missing values
    df = data.drop(cols_2_drop, axis=1)
    return df

def fill_missing_cats(data=None, cat_features=None, missing_encoding=None, missing_col=False):
    '''
    Fill missing values using the mode of the categorical features.
    Parameters:
    ------------------------
        data: DataFrame or name Series.
            Data set to perform operation on.
        cat_features: List, Series, Array.
            categorical features to perform operation on. If not provided, we automatically infer the categoricals from the dataset.
        missing_encoding: List, Series, Array.
            Values used in place of missing. Popular formats are [-1, -999, -99, '', ' ']
        missin_col: bool, Default True
      Creates a new column to capture the missing values. 1 if missing and 0 otherwise. This can sometimes help a machine learning model.
      
      This function is used in the deal_with_missing_value function.
    '''

    if data is None:
        raise ValueError("data: Expecting a DataFrame/ numpy2d array, got 'None'")

    if cat_features is None:
        cat_features = get_cat_feats(data)

    df = data.copy()
    #change all possible missing values to NaN
    if missing_encoding is None:
        missing_encoding = ['', ' ', -99, -999]

    df.replace(missing_encoding, np.NaN, inplace=True)
    
    for feat in cat_features:
        if missing_col:
            df[feat + '_missing_value'] = (df[feat].isna()).astype('int64')
        most_freq = df[feat].mode()[0]
        df[feat] = df[feat].replace(np.NaN, most_freq)
    
    return df

def fill_missing_num(data=None, num_features=None, method='mean', missing_col=False):
    '''
    fill missing values in numerical columns with specified [method] value
    Parameters:
        ------------------------------
        data: DataFrame or name Series.
            The data set to fill
        features: list.
            List of columns to fill
        method: str, Default 'mean'.
            method to use in calculating fill value.
        missing_col: bool, Default True
          Creates a new column to capture the missing values. 1 if missing and 0 otherwise. This can sometimes help a machine learning model.
          
          This function is used in the deal_with_missing_value function.
    '''
    if data is None:
        raise ValueError("data: Expecting a DataFrame/ numpy2d array, got 'None'")
    
    if num_features is None:
        num_features = get_num_feats(data)
        #get numerical features with missing values
        temp_df = data[num_features].isna().sum()
        features = list(temp_df[num_features][temp_df[num_features] > 0].index)
        
    df = data.copy()
    for feat in features:
        if missing_col:
            df[feat + '_missing_value'] = (df[feat].isna()).astype('int64')
        if method is 'mean':
            mean = df[feat].mean()
            df[feat].fillna(mean, inplace=True)
        elif method is 'median':
            median = df[feat].median()
            df[feat].fillna(median, inplace=True)
        elif method is 'mode':
            mode = df[feat].mode()[0]
            df[feat].fillna(mode, inplace=True)
        else:
            raise ValueError("method: must specify a fill method, one of [mean, mode or median]'")
    return df

In [16]:
def deal_with_missing_value(data, percent=70):
    """
    this function automatically take care of missing values.
        It fills the missing values in categorical variables with mode of the particular column
        and fills the missing value numerical variables with mean of the particular column.
        It automatically drops columns with more than 70% missing values except when set otherwise.
        
        This function is used in the feature_preprocessing function to deal with missing values. It can also be used alone.
        """
    if data is None:
        raise ValueError("data: Expecting a DataFrame/ numpy2d array, got 'None'")
    df1 = drop_missing(data=data, percent=percent)
    df2 = fill_missing_cats(data=df1)
    df = fill_missing_num(data=df2)
    return df

In [17]:
def drop_redundant(data):
    '''
    Removes features with the same value in all cell. Drops feature If Nan is the second unique class as well.
    Parameters:
    -----------------------------
        data: DataFrame or named series.
    
    Returns:
        DataFrame or named series.
    This function is used in the feature_processing function.
    '''

    if data is None:
        raise ValueError("data: Expecting a DataFrame/ numpy2d array, got 'None'")
    
    #get columns
    cols_2_drop = _nan_in_class(data)
    print("Dropped {}".format(cols_2_drop))
    df = data.drop(cols_2_drop, axis=1)
    return df
def _nan_in_class(data):
    """helper function for drop_redundant function"""
    cols = []
    for col in data.columns:
        if len(data[col].unique()) == 1:
            cols.append(col)

        if len(data[col].unique()) == 2:
            if np.nan in list(data[col].unique()):
                cols.append(col)

    return cols

### Different Encoding Techniques

In [123]:
#Label Encoding for object to numeric conversion
def binaryencoder(data):
    """To avoid the curse of dimensionality, this function only encodes categorical features with less than 7 unique values
    
        This function can be used alone. It is also used in the encode_data function.
        
        It is the default method for encoding categorical variables with unique value less than four in the encode_data function.
        
        explanation: It is similar to onehot encoding but gives lesser dimensions, making it a better option. 
                    It converts the unique entry into binary combination and then creates column using binary hashing.
    """
    features = get_cat_feats(data=data)
    cols = []
    for feature in features:
        if data[feature].nunique() < 7:
            cols.append(feature)
        
    enc = BinaryEncoder(cols=cols).fit(data)
    data = enc.transform(data)
    return data

def onehotencoder(data):
    """To avoid the curse of dimensionality, this function only encodes categorical features with less than 4 unique values
    
        This function can be used alone. It is also used in the encode_data function
        
    The onehotencoder is only used when the number of unique value is less than four to avoid the curse of dimensionality.
    If encode_data method parameter is set to 'onehotencode' this is what is used in encoding categorical variables with 
    number of unique values less than 4.
    
    explanation: it is used mostly for nominal variables such that a binary combination of the unique values are set as new columns
                    in the dataset.
    """
    features = get_cat_feats(data=data)
    cols = []
    for feature in features:
        if data[feature].nunique() < 4:
            cols.append(feature)
        
    enc = OneHotEncoder(cols=cols).fit(data)
    data = enc.transform(data)
    return data

def labelencoder(data):
    """
    This function can be used alone. It is also used in the encode_data function.
    
    It is used for columns that has more than 3 unique values. Such columns are treated as ordinal variables. 
    
    Explanation: Label encoders are ordinal encoders that encode unique values as continuous intergers.
    
    """
    features = get_cat_feats(data=data)
    for feat in features:
        data[feat] = le.fit_transform(data[feat].astype(str))
    return data

def sumencoder(data):
    """
    This function can be used alone. It is also used in the encode_data function.
    
    The sumencoder is only used when the number of unique value is less than four to avoid the curse of dimensionality.
    If encode_data method parameter is set to 'sumencode' this is what is used in encoding categorical variables with 
    number of unique values less than 4.
    
    explanation: it is similar to one-hot encoding but the difference is that in sum encoding one value is taken as '-1'
                and it is not compared to other value.
    """

    features = get_cat_feats(data=data)
    cols = []
    for feature in features:
        if data[feature].nunique() < 4:
            cols.append(feature)
    enc = SumEncoder(cols = cols).fit_transform(data)
    data = enc
    return data

def catboostencoder(data, target):
    '''Data inputs must not be string
        This function is used alone. It is not called by any other function.
        
        it uses the catboost tree model in properly encoding categorical features.
    
        explanation: a target encoder. It uses the target variable in encoding the categorical variables. 
        It is more accurate than most encoding methods.
    '''
    X = data.drop(target, axis=1)
    y = data[target]
    features = get_cat_feats(data=X)
    enc = CatBoostEncoder(cols=features).fit(X,y)
    data = enc.transform(X, y)
    return data

def hashencoder(data):
    '''
         This function can be used alone. It is also used in the encode_data function.
    
        The hashencoder is only used when the number of unique value is less than four to avoid the curse of dimensionality.
        If encode_data method parameter is set to 'hashencode' this is what is used in encoding categorical variables with 
        number of unique values less than 4.

        explanation: Feature hashing maps each category in a categorical feature to an integer within a predetermined range
    
                        The size of the output dimensions is controlled by the variable n_components.
    '''    
    
    cols = get_cat_feats(data)
    new_col = []
    for i in cols:
        string = str(data[i][0]) + str(data[i][len(data)-1]) 
        flag = re.findall(r'\d+', string)
        if len(flag) > 2:
               if len(flag[0])>2:
                    new_col.append(i)
        
    enc = HashingEncoder(cols=new_col, n_components= 1).fit(data)
    data = enc.transform(data)
    
    return data

def embeddingencoder(data):
     """
        This function is used alone.
        It uses neural network embeddings to encode categorical features. 
         """
     embedding_info = ce.get_embedding_info(data)
     X_encoded,encoders = ce.get_label_encoded_data(data)

     return X_endoded
    

In [209]:
def encode_data(data, method='binary'):
    """
        encodes categorical variables automatically using binary encoding for columns with less than 4 unique values
        then label encode all other variables
        method takes either binary or onehot or sumencode or hashcode. default is binary
        
        this function can be used alone and it also used in the feature_processing function.
        
        
    """
    if method == 'binary':
        data = binaryencoder(data)
    elif method== 'onehot':
        data = onehotencoder(data)
    elif method == 'sumencode':
        data = sumencode(data)
    data = labelencoder(data)
    return data

7

In [34]:
def get_qcut(data=None, col=None, q=None, duplicates='drop', return_type='float64'):
    '''
    Cuts a series into bins using the pandas qcut function
    and returns the resulting bins as a series for merging.
    Parameter:
    -------------
        data: DataFrame, named Series
            Data set to perform operation on.
        col: str
            column to cut/binnarize.
        q: integer or array of quantiles
            Number of quantiles. 10 for deciles, 4 for quartiles, etc. Alternately array of quantiles, e.g. [0, .25, .5, .75, 1.] for quartiles.
        duplicates: Default 'drop',
            If bin edges are not unique drop non-uniques.
        return_type: dtype, Default (float64)
            Dtype of series to return. One of [float64, str, int64]
    
    Returns:
    --------
        Series, 1D-Array
        
        This function is not used in any other function. It can be used alone.
    '''

    temp_df = pd.qcut(data[col], q=q, duplicates=duplicates).to_frame().astype('str')
    #retrieve only the qcut categories
    df = temp_df[col].str.split(',').apply(lambda x: x[0][1:]).astype(return_type)
    
    return df

In [239]:
def feature_processing(data=None, target=None, date_cols= None, percent = 70, method='binary'):
    '''this function does feature engineering such as dealing with missing values, dropping redundant columns,
        encoding categorical variables, creating new variables from the date columns and print out the correlation between the encoded variables and the target variable
        
        Parameters
    ----------
        date_cols: list/series/array
            Date column names in the data set.
        show_categories: bool, default False
            Displays the unique classes and counts in each of the categorical feature in the data set.
        plot_missing: bool, default True
            Plots missing values as a heatmap
        target: the target variable in the dataframe
           The dependent variable in the dataset
        percent: float, Default 70
            Percentage of missing values to be in a column before it is eligible for removal.
        method: the method for encoding variables before label-encoding them.
            Value can be binary, onehot, sumencode
        
    Returns:
    -------
        a dataframe of preprocessed columns. It is best to use this function after the text variables have been preprocessed.
    This function is stand alone used for automatic feature engineering.
    
    It hash encodes the id and set it to be the index of the dataframe.
    '''
    if data is None:
        raise ValueError("data: Expecting a DataFrame/ numpy2d array, got 'None'")
    ###hash encoding id
    cols = get_cat_feats(data)
    new_col = []
    for x in cols:
        if re.search("_id", x.lower()) and data[x].nunique()>3:
            string = str(data[x])
            if bool(re.findall(r'\d+', string)) == True:
                new_col.append(x)
    if len(new_col)>0:
        data.set_index(pd.util.hash_pandas_object(data[new_col]), drop=False, inplace=True)
    data = data.drop(new_col, axis=1)
    data = drop_redundant(data)
    data = deal_with_missing_value(data, percent=70)
    X = data.drop(target, axis =1)
    y = data[target]
    if date_cols is not None:
        data_dates = data[date_cols]
        data_dates = date_processing(data=data_dates, date_cols = date_cols)
        X = data.drop(date_cols, axis=1)
    X = encode_data(X, method=method)
    if data[target].dtype == 'object':
        data[target] = le.fit_transform(data[target].astype(str))
    df = pd.concat([X, data_dates, data[target]], axis=1)
    print('Pearson Correlation')
    print(df.corr()[target])
    _space()
    print('Kendall Correlation')
    print(df.corr(method='kendall')[target])
    _space()
    print('Spearman Correlation')
    print(df.corr(method='spearman')[target])
    df = df[get_num_feats(data=df)]
    df['hashed'] = df.index
    return df

## Scaling and Normalization

In [240]:
def minmaxscaler(data, cols = None):
    '''
    It is scaling method use when you want all the values in column to be between 0 and 1. This is used for parametric models
    like SVM and KNN that calculate euclidean distance or any other form of distance using the magnitude of the values.
    
    -------------
        data: DataFrame, named Series
            Data set to perform operation on. It advisable not to scale/normalise the target variable.
        col: list of str
            columns in form of a list to scale/normalise. If not parsed, it scales/normalises the entire dataframe
        Returns:
    --------
        DataFrame of the scaled/normalised data/columns.
    It is used in the scale_normalise_data. It can also be used alone.
    '''
    
    if cols is not None:
        mm_scaler = MinMaxScaler()
        data[cols] = mm_scaler.fit_transform(data[cols])
       
    else:
        col_names = data.columns
        mm_scaler = MinMaxScaler()
        df_mm = mm_scaler.fit_transform(data)
        data = pd.DataFrame(df_mm, columns=col_names)
    
    return data

In [241]:
def standardscaler(data, cols = None):
    ''' 
    It is a standard normalization technique use when using machine learning models that assusme a normal/gaussian distribution.
    Models like Linear Regression, Gaussian Naive Bayes etc.
    -------------
        data: DataFrame, named Series
            Data set to perform operation on. It advisable not to scale/normalise the target variable.
        col: list of str
            columns in form of a list to scale/normalise. If not parsed, it scales/normalises the entire dataframe
        Returns:
    --------
        DataFrame of the scaled/normalised data/columns.
        
    It is used in the scale_normalise_data. It can also be used alone.
     '''
    if cols is not None:
        
        s_scaler = StandardScaler()
        data[cols] = s_scaler.fit_transform(data[cols])
        
        
    else:
        col_names = data.columns
        s_scaler = StandardScaler()
        df_s = s_scaler.fit_transform(data)
        data = pd.DataFrame(df_s, columns=col_names)
    
    return data

In [242]:
def robustscaler(data, cols = None):
    
    '''
    It is similar to the standardscaler, except that is is better used when there are outliers in the dataset.
    -------------
        data: DataFrame, named Series
            Data set to perform operation on. It advisable not to scale/normalise the target variable.
        col: list of str
            columns in form of a list to scale/normalise. If not parsed, it scales/normalises the entire dataframe
        Returns:
    --------
        DataFrame of the scaled/normalised data/columns.
    
    It is used in the scale_normalise_data. It can also be used alone.
        '''
    
    if cols is not None:
        r_scaler = RobustScaler()
        data[cols] = r_scaler.fit_transform(data[cols])
    else:
        col_names = data.columns
        r_scaler = RobustScaler()
        df_r = r_scaler.fit_transform(data)
        data = pd.DataFrame(df_r, columns=col_names)
    
    return data

In [243]:
def conventional_feat_scaling(data, cols = None, method = 'log', interval=[0,1]):
    
    """
    It uses conventional methodz for scaling variables.
    -------------
        data: DataFrame, named Series
            Data set to perform operation on. It advisable not to scale/normalise the target variable.
        col: list of str
            columns in form of a list to scale/normalise. If cols is not parsed, it won't work.
        method:str
            the method of scaling to be used. It can be 'log','linear', 'clipping' or 'z_score'
        interval: list of int
            this is used by the clipping scaling method. the first element is the min val and the second element is the mex val.
        Returns:
    --------
        DataFrame of the scaled/normalised data.
    It is used alone.
    """
    if cols is not None:
        for i in cols:
            if method == 'log':
                data[i] = np.log((1+data[i])/2)
            if method == 'linear':
                maximum = np.max(data[i])
                data[i] = data[i]/maximum
            if method == 'clipping':
                data[i] = np.clip(data[i], interval[0], interval[1])
            if method == 'z_score':
                data[i] = (data[i] - np.mean(data[i]))/np.std(data[i])
                
    else: 
        print("Input the cols parameter. This function works on the columns parsed in the cols parameter")
    
    return data
            

In [244]:
def scale_normalize_data(data, cols = None, method = 'normalize'):
    '''Normalises or Scale variables in the dataset. 
        NB:1. Normalise all features if the ml model assumes a normal/gaussian distribution.
                Example of such models are linear regression, discriminant analysis (LDA) and Gaussian naive Bayes.
                If statistical test such as t-test, ANOVA and other tests that assume normal distribution is to be carried out, 
                then normalise the data before carryingout the test.
            2. Scale the data if to be in range of -1 and 1 
                if parametric model like K Nearest Neighbour and Support Vector Machines are to be used.
        
        -------------
        data: DataFrame, named Series
            Data set to perform operation on. It advisable not to scale/normalise the target variable.
        col: list of str
            columns in form of a list to scale/normalise. If not parsed, it scales/normalises the entire dataframe
        method: str
            The method can either be normalize, normalize_outlier, or scale. It differes base on the used case. The default is normalise.
        return_type: DataFrame
            
    
    Returns:
    --------
        DataFrame of the scaled/normalised data/columns.
        
    This function is not used in any other function. It can be used alone.
    '''
    data = data[get_num_feats(data=data)]
    if method == 'normalize':
        df = standardscaler(data, cols = cols)
    elif method == 'normalize_outlier':
        df = robustscaler(data, cols = cols)
    elif method == 'scale':
        df = minmaxscaler(data, cols = cols)
    return df


### Test Functions

In [41]:
import pandas as pd
datainput = pd.read_csv(r"C:\Users\Ighdaro Emwinghare\Downloads\data2.csv")
datainput2 = pd.read_csv(r"C:\Users\Ighdaro Emwinghare\Downloads\data1.csv")

''' 
merge and join doesn't work correctly since there are similar columns in both datainput and datainput2. 
The correct way is to concat the two dataframes.

'''

" \nmerge and join doesn't work correctly since there are similar columns in both datainput and datainput2. \nThe correct way is to concat the two dataframes.\n\n"

In [147]:
data = datainput2

In [43]:
describe(data=data, name='', date_cols=['period_week','period_month', 'date'], show_categories=False, plot_missing=False, target='product')

First five data points


,product_id,call_id,calls_account_id,date,channel,is_clm,activity_channel,key_message_id,content_category,content_message_local,...,period_week,period_month,con_size,geo_country,ISO3,ISO,areaAbbvie,geo_area,segment_concat,content_message_concat
0,a001t00000JKjL4AAL,a045J000002aHbrQAE,0011t00000IBwKeAAL,5/6/2020,E-mail,0,NaN,NaN,NaN,NaN,...,5/4/2020,5/1/2020,True,Russia,RUS,RU,EEME&A,EEMEA,B3@Rheum_Science_First_Green,(No message)@(No message)@(No message)
1,a001t00000JKjL4AAL,a041t00000OTN0RAAX,0011t00000IBwKeAAL,3/11/2020,NaN,0,NaN,NaN,NaN,NaN,...,3/9/2020,3/1/2020,True,Russia,RUS,RU,EEME&A,EEMEA,B3@Rheum_Science_First_Green,(No message)@(No message)@(No message)
2,a001t00000JKjL4AAL,a041t00000M6rqaAAB,0011t00000IBwKeAAL,1/14/2020,NaN,0,NaN,NaN,NaN,NaN,...,1/13/2020,1/1/2020,True,Russia,RUS,RU,EEME&A,EEMEA,B3@Rheum_Science_First_Green,(No message)@(No message)@(No message)
3,a001t00000JKjL4AAL,a045J000003Lr7JQAS,0011t00000IBwKeAAL,5/14/2020,E-mail,0,NaN,NaN,NaN,NaN,...,5/11/2020,5/1/2020,True,Russia,RUS,RU,EEME&A,EEMEA,B3@Rheum_Science_First_Green,(No message)@(No message)@(No message)
4,a001t00000JKjL4AAL,a045J000002Y5coQAC,0011t00000IBwKeAAL,4/27/2020,E-mail,0,NaN,NaN,NaN,NaN,...,4/27/2020,4/1/2020,True,Russia,RUS,RU,EEME&A,EEMEA,B3@Rheum_Science_First_Green,(No message)@(No message)@(No message)




Random five data points


,product_id,call_id,calls_account_id,date,channel,is_clm,activity_channel,key_message_id,content_category,content_message_local,...,period_week,period_month,con_size,geo_country,ISO3,ISO,areaAbbvie,geo_area,segment_concat,content_message_concat
20685,a001v00002KP02YAAT,a041v00001CjxAkAAJ,001G000001VJ3inIAD,2/20/2020,Face to Face,1,NaN,a0D1v00000NqTKMEA3,(No message),(No message),...,2/17/2020,2/1/2020,True,United Kingdom,GBR,GB,WE&C,WEC,(No segment)@(No segment),(No message)@(No message)@(No message)
38913,a001v00002W5LyvAAF,a041v00001BPqzsAAD,001G000001VJU81IAH,1/10/2020,Face to Face,1,NaN,a0D1v00000NqVytEAF,NaN,NaN,...,1/6/2020,1/1/2020,True,United Kingdom,GBR,GB,WE&C,WEC,(No segment)@(No segment),(No message)@(No message)@(No message)
168517,a001v00002W6XH7AAN,a041v00001ILey0AAD,0010Y00000MAPrAQAX,6/19/2020,Face to Face,1,NaN,a0D1v00000NrHBlEAN,Patient Support Programs,AbbVie Care,...,6/15/2020,6/1/2020,True,Germany,DEU,DE,WE&C,WEC,(No segment)@(No segment),Patient Support Programs@AbbVie Care is a PSP ...
13247,a001v00002KP02YAAT,a041v00001CnWPJAA3,001G000001t4y6WIAQ,3/5/2020,Face to Face,1,NaN,a0D1v00000NqhvcEAB,Safety,RINVOQ+MTX zeigt ein vergleichbares Sicherheit...,...,3/2/2020,3/1/2020,True,Germany,DEU,DE,WE&C,WEC,B3@Rheum_Science First_Green,Safety@Safety profile: robust trial program wi...
130504,a001v00002W6XH7AAN,a041v00001IKuXBAA1,001G000001t4h4oIAA,6/15/2020,Face to Face,1,NaN,a0D1v00000NrHBkEAN,Patient Support Programs,AbbVie Care,...,6/15/2020,6/1/2020,True,Germany,DEU,DE,WE&C,WEC,(No segment)@(No segment),Patient Support Programs@AbbVie Care is a PSP ...




Last five data points


...

,Feature,Unique Count
0,product_id,4
1,call_id,71885
2,calls_account_id,14791
3,date,195
4,channel,6
5,key_message_id,1259
6,content_category,8
7,content_message_local,123
8,local_key_message_vault_id,166
9,content_message_global,17




Missing Values in Data


,features,missing_counts,missing_percent
0,product_id,0,0.0
1,call_id,0,0.0
2,calls_account_id,1674,1.0
3,date,0,0.0
4,channel,8898,5.3
5,is_clm,0,0.0
6,activity_channel,168533,100.0
7,key_message_id,43383,25.7
8,content_category,55232,32.8
9,content_message_local,55229,32.8




Pearson Correlation
                  is_clm  activity_channel  con_size
is_clm               1.0               NaN       NaN
activity_channel     NaN               NaN       NaN
con_size             NaN               NaN       NaN


Kendall Correlation
                  is_clm  activity_channel  con_size
is_clm               1.0               NaN       NaN
activity_channel     NaN               NaN       NaN
con_size             NaN               NaN       1.0


Spearman Correlation
                  is_clm  activity_channel  con_size
is_clm               1.0               NaN       NaN
activity_channel     NaN               NaN       NaN
con_size             NaN               NaN       NaN


Bivariant Stats between categorical features
Number of unique values too large
Number of unique values too large
None


Bivariant Stats between cat feats and target variable
Frequency table
product             ABC_Rinvoq_Rheum  Rinvoq_RA  Rinvoq_RA_KAM
product_id                                

Frequency table
product     ABC_Rinvoq_Rheum  Rinvoq_RA  Rinvoq_RA_KAM
change_clm                                            
CLM                     2628      97945           2761
No CLM                  1958      62049           1192
ChiSquare test statistic:  153.00476497449955
p-value:  5.962624130604463e-34


Frequency table
product                 ABC_Rinvoq_Rheum  Rinvoq_RA  Rinvoq_RA_KAM
touchpoint_channel_clm                                            
(No channel)@0                         0       4328              0
(No channel)@1                         0       4570              0
E-mail@0                             425      24800            251
E-mail@1                               5        244              4
Engage Call@0                          0        144              0
Engage Call@1                          0      11737              0
Face to Face@0                       890      19768            645
Face to Face@1                      2609      77613           274

In [245]:
data1 = feature_processing(data=data, target='product', date_cols=['period_week','period_month', 'date'], percent = 70)

Dropped ['activity_channel', 'indication', 'therapeutic_area', 'measure_source', 'con_size', 'hashed_id']
Dropped []


C:\Users\Ighdaro Emwinghare\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\Ighdaro Emwinghare\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Ighdaro Emwinghare\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

Pearson Correlation
channel_0                      NaN
channel_1                 0.006167
channel_2                 0.000633
channel_3                -0.019481
is_clm_0                  0.028209
is_clm_1                 -0.028209
content_category         -0.039078
content_message_local     0.027485
content_message_global    0.055251
id                       -0.057671
call_date                -0.079067
product_0                      NaN
product_1                -0.072238
product_2                -0.691198
fs_product_id_0          -0.008502
fs_product_id_1           0.008502
segment_quant             0.015959
segment_qual             -0.000590
change_clm_0              0.028209
change_clm_1             -0.028209
touchpoint_channel_clm    0.000283
geo_country               0.181180
ISO3                      0.089050
ISO                       0.088912
areaAbbvie_0             -0.009004
areaAbbvie_1              0.007493
areaAbbvie_2              0.006036
geo_area_0               -0.009004


In [252]:
X = data1.drop('product', axis=1)
y = data1['product']

In [253]:
scale_normalize_data(X, method='normalize_outlier')

,channel_0,channel_1,channel_2,channel_3,is_clm_0,is_clm_1,content_category,content_message_local,content_message_global,id,...,day_period_week,dayweek_period_week,month_period_month,year_period_month,day_period_month,dayweek_period_month,month_date,year_date,day_date,dayweek_date
0,0.0,0.0,-1.0,1.0,-1.0,1.0,0.0,0.0,0.0,-0.383018,...,-0.785714,0.0,0.666667,0.0,0.0,0.000000,0.666667,0.0,-0.714286,0.0
1,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,-0.383018,...,-0.428571,0.0,0.000000,0.0,0.0,0.666667,0.000000,0.0,-0.357143,0.0
2,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,-0.383018,...,-0.142857,0.0,-0.666667,0.0,0.0,-0.666667,-0.666667,0.0,-0.142857,-0.5
3,0.0,0.0,-1.0,1.0,-1.0,1.0,0.0,0.0,0.0,-0.383018,...,-0.285714,0.0,0.666667,0.0,0.0,0.000000,0.666667,0.0,-0.142857,0.5
4,0.0,0.0,-1.0,1.0,-1.0,1.0,0.0,0.0,0.0,-0.383018,...,0.857143,0.0,0.333333,0.0,0.0,-0.666667,0.333333,0.0,0.785714,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168528,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.417298,...,0.642857,0.0,-0.333333,0.0,0.0,0.333333,-0.333333,0.0,0.642857,-0.5
168529,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,-0.383018,...,0.142857,0.0,-0.333333,0.0,0.0,0.333333,-0.333333,0.0,0.214286,0.0
168530,0.0,0.0,0.0,1.0,-1.0,1.0,0.0,0.0,0.0,-0.383018,...,0.571429,0.0,0.000000,0.0,0.0,0.666667,0.000000,0.0,0.571429,-0.5
168531,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,-0.383018,...,0.642857,0.0,-0.333333,0.0,0.0,0.333333,-0.333333,0.0,0.714286,0.0


In [254]:
scale_normalize_data(X, method='normalize')

,channel_0,channel_1,channel_2,channel_3,is_clm_0,is_clm_1,content_category,content_message_local,content_message_global,id,...,day_period_week,dayweek_period_week,month_period_month,year_period_month,day_period_month,dayweek_period_month,month_date,year_date,day_date,dayweek_date
0,0.0,-0.320656,-1.751205,1.506802,-1.258929,1.258929,-0.147863,0.205569,0.550529,-0.943900,...,-1.240995,0.0,0.915655,0.098946,0.0,0.289538,0.915655,0.098946,-1.173219,0.002213
1,0.0,-0.320656,0.571035,-0.663657,-1.258929,1.258929,-0.147863,0.205569,0.550529,-0.943900,...,-0.643010,0.0,-0.110266,0.098946,0.0,1.261487,-0.110266,0.098946,-0.579732,0.002213
2,0.0,-0.320656,0.571035,-0.663657,-1.258929,1.258929,-0.147863,0.205569,0.550529,-0.943900,...,-0.164622,0.0,-1.136188,0.098946,0.0,-0.682411,-1.136188,0.098946,-0.223640,-0.709501
3,0.0,-0.320656,-1.751205,1.506802,-1.258929,1.258929,-0.147863,0.205569,0.550529,-0.943900,...,-0.403816,0.0,0.915655,0.098946,0.0,0.289538,0.915655,0.098946,-0.223640,0.713927
4,0.0,-0.320656,-1.751205,1.506802,-1.258929,1.258929,-0.147863,0.205569,0.550529,-0.943900,...,1.509736,0.0,0.402694,0.098946,0.0,-0.682411,0.402694,0.098946,1.319426,-1.421215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168528,0.0,-0.320656,0.571035,-0.663657,0.794326,-0.794326,-0.147863,0.205569,0.550529,0.507964,...,1.150945,0.0,-0.623227,0.098946,0.0,0.775513,-0.623227,0.098946,1.082031,-0.709501
168529,0.0,-0.320656,0.571035,-0.663657,-1.258929,1.258929,-0.147863,0.205569,0.550529,-0.943900,...,0.313766,0.0,-0.623227,0.098946,0.0,0.775513,-0.623227,0.098946,0.369847,0.002213
168530,0.0,-0.320656,0.571035,1.506802,-1.258929,1.258929,-0.147863,0.205569,0.550529,-0.943900,...,1.031348,0.0,-0.110266,0.098946,0.0,1.261487,-0.110266,0.098946,0.963334,-0.709501
168531,0.0,-0.320656,0.571035,-0.663657,-1.258929,1.258929,-0.147863,0.205569,0.550529,-0.943900,...,1.150945,0.0,-0.623227,0.098946,0.0,0.775513,-0.623227,0.098946,1.200728,0.002213


In [255]:
scale_normalize_data(X, method='scale')

,channel_0,channel_1,channel_2,channel_3,is_clm_0,is_clm_1,content_category,content_message_local,content_message_global,id,...,day_period_week,dayweek_period_week,month_period_month,year_period_month,day_period_month,dayweek_period_month,month_date,year_date,day_date,dayweek_date
0,0.0,0.0,0.0,1.0,0.0,1.0,0.333333,0.595041,0.733333,0.120866,...,0.100000,0.0,0.363636,1.0,0.0,0.666667,0.363636,1.0,0.166667,0.333333
1,0.0,0.0,1.0,0.0,0.0,1.0,0.333333,0.595041,0.733333,0.120866,...,0.266667,0.0,0.181818,1.0,0.0,1.000000,0.181818,1.0,0.333333,0.333333
2,0.0,0.0,1.0,0.0,0.0,1.0,0.333333,0.595041,0.733333,0.120866,...,0.400000,0.0,0.000000,1.0,0.0,0.333333,0.000000,1.0,0.433333,0.166667
3,0.0,0.0,0.0,1.0,0.0,1.0,0.333333,0.595041,0.733333,0.120866,...,0.333333,0.0,0.363636,1.0,0.0,0.666667,0.363636,1.0,0.433333,0.500000
4,0.0,0.0,0.0,1.0,0.0,1.0,0.333333,0.595041,0.733333,0.120866,...,0.866667,0.0,0.272727,1.0,0.0,0.333333,0.272727,1.0,0.866667,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168528,0.0,0.0,1.0,0.0,1.0,0.0,0.333333,0.595041,0.733333,0.553421,...,0.766667,0.0,0.090909,1.0,0.0,0.833333,0.090909,1.0,0.800000,0.166667
168529,0.0,0.0,1.0,0.0,0.0,1.0,0.333333,0.595041,0.733333,0.120866,...,0.533333,0.0,0.090909,1.0,0.0,0.833333,0.090909,1.0,0.600000,0.333333
168530,0.0,0.0,1.0,1.0,0.0,1.0,0.333333,0.595041,0.733333,0.120866,...,0.733333,0.0,0.181818,1.0,0.0,1.000000,0.181818,1.0,0.766667,0.166667
168531,0.0,0.0,1.0,0.0,0.0,1.0,0.333333,0.595041,0.733333,0.120866,...,0.766667,0.0,0.090909,1.0,0.0,0.833333,0.090909,1.0,0.833333,0.333333


### Notes.

1. Added New functions for scaling and normalising dataset. Then a master function 'scale_normalize_data' that use standard_scaler for normalization as default. It can also use robust_scaler when method is set to 'normalize_outlier'. It also uses minmax_scalere for scaling when method is set to'scale'. 
2. Made change to the feature_processing function to only output numeric columns as part of the dataframe. 

3. The 3 master functions are **describe**, **feature_processing** and **scale_normalize_data** functions.
4. The hashing of the ids is done by the **feature_processing** function.